# 4주차 : 

In [1]:
# HuggingFace Transformers 라이브러리를 설치합니다.
!pip install -q transformers

     |████████████████████████████████| 4.9 MB 7.3 MB/s 
     |████████████████████████████████| 163 kB 82.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 


In [21]:
from transformers import AutoModelForSequenceClassification, Trainer, AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
id2label = {0: '관광지 추천', 1: '날씨 묻기', 2: '맛집 추천', 3: '숙소 추천', 4: '인사', 5: '카페 추천', 6: '활동 추천'}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/812449f1a6bc736e693db7aa0e513e5e90795a62/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/812449f1a6bc736e693db7aa0e513e5e90795a62/vocab.txt
loading file tokenizer.json from cache at /root/.cache/hu

In [22]:
# 저장한 모델을 불러오기 위해 구글 드라이브에 마운트합니다.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# 모델 불러오기 (경로 수정 필요 -> 모델 관련 파일들을 모아놓은 폴더의 경로)
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/중정처1주차/my_model", local_files_only=True)
trainer = Trainer(model=model)

loading configuration file /content/drive/MyDrive/중정처1주차/my_model/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/\u110c\u116e\u11bc\u110c\u1165\u11bc\u110e\u11651\u110c\u116e\u110e\u1161/my_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\uad00\uad11\uc9c0 \ucd94\ucc9c",
    "1": "\ub0a0\uc528 \ubb3b\uae30",
    "2": "\ub9db\uc9d1 \ucd94\ucc9c",
    "3": "\uc219\uc18c \ucd94\ucc9c",
    "4": "\uc778\uc0ac",
    "5": "\uce74\ud398 \ucd94\ucc9c",
    "6": "\ud65c\ub3d9 \ucd94\ucc9c"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\uad00\uad11\uc9c0 \ucd94\ucc9c": 0,
    "\ub0a0\uc528 \ubb3b\uae30": 1,
    "\ub9db\uc9d1 \ucd94\ucc9c": 2,
    "\uc219\uc18c \ucd94\ucc9c": 3,
    "\uc778\uc0ac": 4,
    "\uce74\ud398 \ucd94\ucc

In [24]:
def inference(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)
  logits = outputs.logits

  probs = logits.squeeze().cpu()
  predictions = np.where(probs==probs.max(), 1, 0)
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

  return predicted_labels[0]

In [25]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [26]:
# 다음의 명령어로 ngrok linux version 을 설치

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2022-10-01 06:34:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.1’

ngrok-stable-linux- 100%[===================>]  13.13M  17.3MB/s    in 0.8s    

2022-10-01 06:34:37 (17.3 MB/s) - ‘ngrok-stable-linux-amd64.tgz.1’ saved [13770165/13770165]



In [27]:
# 압축 해제

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [28]:
# ngrok 사이트 (https://dashboard.ngrok.com/get-started/setup)에서 회원가입을 한뒤
# "Your Authtoken" 탭을 클릭하여 authtoken을 복사하여 붙여넣습니다.

!./ngrok authtoken 2Esv8w22whjYE91p5YvcBtyNdKq_4nuor1sKynpk11JqT391q

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [29]:
# 플라스크 임포트
from flask import Flask
from flask import render_template, request
from flask_ngrok import run_with_ngrok

In [30]:
# softmax 와 threshold 를 사용하는 새로운 인퍼런스 함수
import torch
import torch.nn.functional as F

def inference_with_softmax(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)

  logits = outputs.logits

  probs = logits.squeeze().cpu()
  softmax_probs = F.softmax(probs, dim=-1)
  labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]

  if softmax_probs[labels_predict] < 0.97:
    return '기타'
  else:
    return id2label[labels_predict]

In [31]:
import random
def make_sentence(intention):
  if intention == "관광지 추천":
    weather = ['봄', '여름', '가을', '겨울']
    choice_weather = random.choice(weather)

    attractions = ['감천 문화마을', '해운대', '태종대', '광안리 해수욕장', '자갈치 시장', '해운대 달맞이길', '송도 해수욕장']
    choice_attractions = random.choice(attractions)

    sentence = f'이번 {choice_weather}, {choice_attractions} 가 보시는 건 어때요?'
    
    return sentence
  
  if intention == "날씨 묻기":
    day = ['맑습니다.', '흐립니다.', '비가 내립니다.', '눈이 옵니다.', '화창합니다.']
    choice_day = random.choice(day)

    sentence = f'오늘 날씨는 {choice_day}'
    
    return sentence

  if intention == "맛집 추천":
    food = ["톤쇼우", "해목", "신발원", "할매국밥", "평산옥"]
    choice_food = random.choice(food)

    sentence = f'부산 {choice_food}는(은) 어떠세요?'

    return sentence

  if intention == "숙소 추천":
    house = ["한화리조트 해운대", "어반스테이 서면", "호메르스 호텔", "하운드호텔 해운대", "호텔그레이 남포", "신라스테이 해운대"]
    choice_house = random.choice(house)

    sentence = f'부산 {choice_house}는(은) 어떠세요?'

    return sentence

  if intention == "카페 추천":
    taste = ["디저트가 맛있는", "음료가 맛있는", "커피가 맛있는", "메뉴가 다양한", "전망이 좋은"]
    choice_taste = random.choice(taste)

    cafe = ["비비비당", "블랙업 커피", "흰여울비치", "카페 덕미", "구름에"]
    choice_cafe = random.choice(cafe)

    sentence = f'{choice_taste} 부산 {choice_cafe}는(은) 어떠세요?'

    return sentence

  if intention == "활동 추천":
    together = ["가족과 함께", "친구와 함께", "애인과 함께"]
    choice_together = random.choice(together)

    activity = ["부산 어린이 대공원", "다이빙", "요트", "패러글라이딩", "해상 케이블카"]
    choice_activity = random.choice(activity)

    sentence = f'{choice_together} {choice_activity}는(은) 어떠세요?'

    return sentence

In [36]:
def answer_sentence(text):

  ####### 훈련한 모델을 이용하여 예측하기 #######
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = logits.squeeze().cpu()

  # 소프트맥스를 활용하여 logit의 값을 확률값으로 정리
  softmax_probs = F.softmax(probs, dim=-1)

  # 확률값 중 가장 큰 값의 인덱스를 구합니다.
  biggest = np.where(softmax_probs==softmax_probs.max())[0][0]
  pred = id2label[biggest]

  # threshold 지정해주기
  threshold = 0.1
  if softmax_probs[biggest] < threshold:
    print("probability is too low")
    pred = ""
  else:
    pass

  ####### 주제에 따라 다른 답변 문장 만들어내기 #######
  if pred == "날씨 묻기":
    ans = make_sentence("날씨 묻기")
    #print(ans)
    return ans

  if pred == "관광지 추천": # ✓
    ans = make_sentence("관광지 추천")
    #print(ans)
    return ans

  if pred == "숙소 추천":
    ans = make_sentence("숙소 추천")
    #print(ans)
    return ans

  if pred == "맛집 추천":
    ans = make_sentence("맛집 추천")
    #print(ans)
    return ans

  if pred == "카페 추천":
    ans = make_sentence("카페 추천")
    #print(ans)
    return ans

  if pred == "활동 추천":
    ans = make_sentence("활동 추천")
    #print(ans)
    return ans

  else:
    print("제가 대답하지 못하는 분야입니다 ㅜㅜ")
    return False

In [37]:
# 입력 : 사용자가 입력한 텍스트
# 출력 : 의도에 대한 답변

def answer(text):
  
  pred = inference_with_softmax(text)
  # pred = inference(text)
  
  # id2label.values() : dict_values(['날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '활동 추천', '카페 추천'])
  if pred in id2label.values():
    return (True, answer_sentence(pred))
  else:
    return (False, answer_sentence(pred))

In [38]:
def answer_system(input_text):
  
  print("안녕하세요, 무엇을 도와드릴까요?")
  _, sentence = answer(input_text)
  
  return sentence 

In [39]:
from flask import Flask, url_for, redirect, render_template, request

TEMPLATE_FOLDER = '/content/drive/MyDrive/중정처 4주차 Web/06/templates'
STATIC_FOLDER = '/content/drive/MyDrive/중정처 4주차 Web/06/static'
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat') 
def chatbot():
    return render_template('chat.html')

@app.route('/get')
def get_bot_response():
    userText = request.args.get('a')
    return answer_system(userText)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9f80-35-230-29-64.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/js/slideshow.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/images/photo-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/images/photo-3.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/images/img-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/images/img-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/images/img-3.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:24] "GET /static/images/sns-1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:25] "GET /static/images/sns-2.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:39:25] "G

안녕하세요, 무엇을 도와드릴까요?


INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 06:40:16] "GET /get?a=카페 HTTP/1.1" 200 -


안녕하세요, 무엇을 도와드릴까요?
